### Please provide your own S3 bucket below. The name for your bucket must contain the prefix ‘deeplens’. In this example, the bucket is ‘deeplens-imageclassification’. Make Sure S3 bucket name is unique, e.g. deeplens-imageclassfication-name-date

In [1]:
bucket='deeplens-imageclassification-matthew'

In [5]:
def upload_to_s3(channel, file):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = channel + '/' + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)

In [2]:
!aws s3 cp s3://deeplens-imageclassification-matthew/data ./data --recursive

In [3]:
# data/data表示生成的".lst"文件的保存路径和文件名前缀,比如data_train.lst和data_val.lst
#data/train表示要处理图像所在路径
#--list表示接下来的执行会生成.lst文件
#--recursive表示迭代搜索图像路径
#--train-ratio,表示数据化分比例,0.9表示将数据划分10份,训练集:验证集=9:1
!python im2rec.py data/data data/train --list --recursive --train-ratio 0.9


cat 0
dog 1


In [4]:
#--num-thread 8 代表采用8线程来生成RecordIO文件
#data/data_train.lst或data/data_val.lst表示要处理的.lst文件所在路径.
#data/train表示图像所在路径
!python im2rec.py --num-thread 32 data/data_train.lst data/train
!python im2rec.py --num-thread 32 data/data_val.lst data/train

Creating .rec file from /home/ec2-user/SageMaker/deeplens/data/data_train.lst in /home/ec2-user/SageMaker/deeplens/data
time: 0.07648539543151855  count: 0
time: 3.523789644241333  count: 1000
time: 3.3782706260681152  count: 2000
time: 2.662682056427002  count: 3000
time: 2.8794918060302734  count: 4000
time: 2.942836284637451  count: 5000
time: 2.9851672649383545  count: 6000
time: 5.00987434387207  count: 7000
time: 3.1225271224975586  count: 8000
time: 3.1663126945495605  count: 9000
time: 3.1464602947235107  count: 10000
time: 2.96248722076416  count: 11000
time: 2.95965838432312  count: 12000
time: 2.634810447692871  count: 13000
time: 5.036537170410156  count: 14000
time: 2.23531436920166  count: 15000
time: 2.8745625019073486  count: 16000
time: 3.0861454010009766  count: 17000
time: 2.786839008331299  count: 18000
time: 2.819965362548828  count: 19000
time: 4.932088136672974  count: 20000
time: 2.889134645462036  count: 21000
time: 2.2618253231048584  count: 22000
Creating .re

In [ ]:
!mv data/data_train.rec ./
!mv data/data_val.rec ./

In [ ]:
upload_to_s3('kaggle/train', 'data_train.rec')


In [2]:
#test rec generating
!python im2rec.py -h

usage: im2rec.py [-h] [--list] [--exts EXTS [EXTS ...]] [--chunks CHUNKS]
                 [--train-ratio TRAIN_RATIO] [--test-ratio TEST_RATIO]
                 [--recursive] [--no-shuffle] [--pass-through]
                 [--resize RESIZE] [--center-crop] [--quality QUALITY]
                 [--num-thread NUM_THREAD] [--color {-1,0,1}]
                 [--encoding {.jpg,.png}] [--pack-label]
                 prefix root

Create an image list or make a record database by reading from an image list

positional arguments:
  prefix                prefix of input/output lst and rec files.
  root                  path to folder containing images.

optional arguments:
  -h, --help            show this help message and exit

Options for creating image lists:
  --list                If this is set im2rec will create image list(s) by
                        traversing root folder and output to <prefix>.lst.
                        Otherwise im2rec will read <prefix>.lst and create a
         